In [2]:
import pandas as pd
from utils import clean_company_name, clean_dataframe 

In [11]:
def get_common_names(companies):
    companies = clean_dataframe(companies,[])
    companies= companies.applymap(clean_company_name)
    return list(set(companies.iloc[:, 0]).intersection(set(companies.iloc[:, 1])))

In [24]:
df_complaints = pd.read_csv('complaints.csv', chunksize=1000, keep_default_na=False, na_values=['NA'])
df_sub = pd.read_csv("sub.txt", sep='\t', chunksize=1000, keep_default_na=False, na_values=['NA'])

complaint_chunks = []
sub_chunks = []

for complaint_chunk, sub_chunk in zip(df_complaints, df_sub):
    common_names = get_common_names(pd.concat([complaint_chunk['Company'], sub_chunk['name']], axis=1))
    complaint_chunk['Company'] = complaint_chunk['Company'].apply(clean_company_name)
    sub_chunk['name'] = sub_chunk['name'].apply(clean_company_name)
    complaint_chunk = complaint_chunk[complaint_chunk['Company'].isin(common_names)]
    sub_chunk = sub_chunk[sub_chunk['name'].isin(common_names)]
    
    # Append concatenated chunks to lists
    complaint_chunks.append(complaint_chunk)
    sub_chunks.append(sub_chunk)

# Concatenate all chunks into single DataFrames
df_complaints_concatenated = pd.concat(complaint_chunks)
df_sub_concatenated = pd.concat(sub_chunks)

# Save concatenated DataFrames to CSV files
df_complaints_concatenated.to_csv('complaints_concatenated.csv', index=False)
df_sub_concatenated.to_csv('sub_concatenated.csv', index=False)

In [25]:
df_complaints_concatenated['Complaint ID'].count()

2140

In [26]:
df_sub_concatenated['cik'].count()

60